<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/PydanticOutputParser_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
%%capture --no-stderr

# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')
MISTRAL_API_KEY=userdata.get('MISTRAL_API_KEY')
TOGETHER_API_KEY=userdata.get('TOGETHER_API_KEY')
ANTHROPIC_API_KEY=userdata.get('ANTHROPIC_API_KEY')
DEEPSEEK_API_KEY=userdata.get('DEEPSEEK_API_KEY')


# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
%pip install langchain -qU  # Instalar la librería principal de LangChain.


# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
%pip install langchain-openai -qU
%pip install langchain-groq -qU
%pip install langchain-google-genai -qU
%pip install langchain-huggingface -qU
%pip install langchain_mistralai -qU
%pip install langchain-together -qU
%pip install langchain-anthropic -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint
from langchain_mistralai import ChatMistralAI
from langchain_together import ChatTogether
from langchain_anthropic import ChatAnthropic

from pprint import pprint

In [44]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

# Modelo Pydantic para respuesta simple
class TranslationOutput(BaseModel):
    translation: str = Field(description="The professional translated text")

# Modelo Pydantic para respuesta de un modelo corrector
class RespuestaAgente (BaseModel):
    translation: str = Field(description="Texto con la traduccion generada por el agente.")
    suggestions: List[str] = Field(description="Lista de suggestions encontradas en el texto.")

parser_translation = PydanticOutputParser(pydantic_object=TranslationOutput)
parser_agent = PydanticOutputParser(pydantic_object=RespuestaAgente)



In [45]:
translator_prompt_template = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a senior legal translator specializing in Intellectual Property documents. "
        "Translate the text from {source_language} to {target_language} with:\n"
        "- Perfect accuracy\n"
        "- Legal terminology consistency\n"
        "- Publication-ready quality\n\n"
        "**Strict instructions:**\n"
        "1. Output MUST be valid JSON with double quotes\n"
        "2. Follow EXACTLY this structure:\n"
        "3. Do not include any additional text or explanations."
    )),
    ("human", (
        "Translate this legal text with absolute precision:\n"
        "{original_text}\n\n"
        "Output ONLY the required JSON:"
    ))
])

# Configurar las variables de entrada
translator_prompt_template = translator_prompt_template.partial(
    format_instructions=parser_translation.get_format_instructions()
)




accuracy_reviewer_prompt_template = ChatPromptTemplate.from_messages([
    ("system", (
        "You are an Accuracy Reviewer specializing in {source_language} to {target_language} translations.\n"
        "**Strict instructions:**\n"
        "1. Follow EXACTLY this structure:\n"
        "2. Return a JSON object with EXACTLY these fields:\n"
        "   - 'translation': corrected translation text\n"
        "   - 'suggestions': list of suggestions using format '- ERROR: [issue] -> SUGGESTION: [fix]'\n\n"
        "3. Follow these rules:\n"
        "- Correct only accuracy errors (mistranslations, omissions, untranslated text)\n"
        "- Maintain original style and format\n"
        "- If no changes, return original text and empty list\n"
        "- Use exactly the specified JSON structure"
    )),
    ("human", (
        "Original Text ({source_language}):\n"
        "{original_text}\n\n"
        "Current Translation ({target_language}):\n"
        "{translation}\n\n"
        "Provide corrections in the required JSON format."
    ))
])

# Configuración adicional para el parser
accuracy_reviewer_prompt_template = accuracy_reviewer_prompt_template.partial(
    format_instructions=parser_agent.get_format_instructions()
)




In [46]:
# llm = ChatOpenAI(model="gpt-4o-mini",api_key=OPENAI_API_KEY, temperature=1)
llm =  ChatGroq(model="deepseek-r1-distill-llama-70b", api_key=GROQ_API_KEY, temperature=1.3)

El modelo recibira un texto original, una traduccion y debe realizar una correccion de la traduccion.
Como resultado debe devolver una nueva traduccion y una lista con los problema encontrados en cierto formato

In [47]:
original_text='And according to Bertrand Russell, "the man who has no tincture of philosophy goes through life imprisoned in the prejudices derived from common sense, from the habitual beliefs of his age or his nation, and from convictions which have grown up in his mind without the cooperation or consent of his deliberate reason.'

In [48]:
translator_prompt = translator_prompt_template.format(source_language="English",
                                         target_language="Spanish",
                                         original_text=original_text,
                                         )

In [49]:
from langchain_core.runnables import RunnablePassthrough

translator_chain = translator_prompt_template | llm | parser_translation
accuracy_reviewer_chain = accuracy_reviewer_prompt_template | llm | parser_agent


full_chain = RunnablePassthrough.assign(translation=translator_chain) | accuracy_reviewer_chain

# Esto no puede ser
refull_chain =  translator_prompt_template | llm | parser_translation | accuracy_reviewer_prompt_template | llm | parser_agent

In [41]:
resultado_final = full_chain.invoke({
    "source_language": "English",
    "target_language": "Spanish",
    "original_text": original_text
})

Como hacerlo sin encadenar todo en una sola cadena

In [53]:
result = translator_chain.invoke({
        "source_language": "English",
        "target_language": "Spanish",
        "original_text": original_text
    })

print(type(result))
print(result.translation)

<class '__main__.TranslationOutput'>
Y según Bertrand Russell, "el hombre que no tiene ninguna tintura de filosofía atraviesa la vida encerrado en los prejuicios derivados del sentido común, en las creencias habituales de su época o su nación, y en las convicciones que han crecido en su mente sin la cooperación o consentimiento de su razón deliberada.


In [58]:
result2 = accuracy_reviewer_chain.invoke({
        "source_language": "English",
        "target_language": "Spanish",
        "original_text": original_text,
        "translation": result.translation
    })

pprint(type(result2))
pprint(result2.translation)
for suggestion in result2.suggestions:
    print(suggestion)

<class '__main__.RespuestaAgente'>
('Y según Bertrand Russell, "el hombre que no tiene tintura de filosofía '
 'atraviesa la vida encerrado en los prejuicios derivados del sentido común, '
 'de las creencias habituales de su época o su nación, y en las convicciones '
 'que se han desarrollado en su mente sin la cooperación o consentimiento de '
 'su razón deliberada.')
- ERROR: The phrase 'no tiene ninguna tintura' is redundant. -> SUGGESTION: Simplify to 'no tiene tintura'
- ERROR: 'han crecido' is not entirely accurate for 'have grown up in his mind' -> SUGGESTION: Change to 'se han desarrollado'
- ERROR: The pronoun 'su' before 'razón deliberada' is redundant -> SUGGESTION: Remove 'su'


crear graficos que explican el sistema encadenado, el no encadenado
(lo que entra y lo que sale)

y tambien la arquitectura lineal
y la que podria ser RunnableParallel
